# Introduction

A startup called Sparkify wants to analyze the data they've been collecting on songs and user activity on their new music streaming app. The analytics team is particularly interested in understanding what songs users are listening to. Currently, they don't have an easy way to query their data, which resides in a directory of JSON logs on user activity on the app, as well as a directory with JSON metadata on the songs in their app.

They'd like a data engineer to create a Postgres database with tables designed to optimize queries on song play analysis, and bring you on the project. Your role is to create a database schema and ETL pipeline for this analysis. You'll be able to test your database and ETL pipeline by running queries given to you by the analytics team from Sparkify and compare your results with their expected results.


# Project Description
In this project, I will showcase data modeling with Postgres and build an ETL pipeline using Python.

## Other important files
> [README](https://github.com/ericagreene87/Udacity-Data-Analytics-Nanodegree/blob/main/Sparkify%20README%20(1).md) Provides discusstion about project

> [create_tables](https://github.com/ericagreene87/Udacity-Data-Analytics-Nanodegree/blob/main/sparkify_create_tables%20.py) Contains link to database and detailed instructions on the ETL process for each of the tables.

> [sql_queries](https://github.com/ericagreene87/Udacity-Data-Analytics-Nanodegree/blob/main/Sparkify%20sql_queries.py) Provides SQL queries 

> [etl.py](https://github.com/ericagreene87/Udacity-Data-Analytics-Nanodegree/blob/main/Sparkify%20etl.py) Reads and processes a single file from song and log data and loads the data into tables


In [1]:
run create_tables.py

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`

In [5]:
song_files = get_files('data/song_data')

In [6]:
filepath = song_files[0]

In [7]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR0IAWL1187B9A96D0,8.4177,Panama,-80.11278,Danilo Perez,197.19791,1,SONSKXP12A8C13A2C9,Native Soul,2003


#### Extract Data for Songs Table and Convert to List

In [8]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
song_data

['SONSKXP12A8C13A2C9', 'Native Soul', 'AR0IAWL1187B9A96D0', 2003, 197.19791]

#### Insert Record into Song Table

In [9]:
cur.execute(song_table_insert, song_data)
conn.commit()

#### Extract Data for Artists Table and Convert to List

In [10]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data

['AR0IAWL1187B9A96D0', 'Danilo Perez', 'Panama', 8.4177, -80.11278]

#### Insert Record into Artist Table

In [11]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

# Process `log_data`

In [12]:
log_files = get_files('data/log_data/')

In [13]:
filepath = log_files[0]

In [14]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Mitch Ryder & The Detroit Wheels,Logged In,Tegan,F,65,Levine,205.03465,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Jenny Take A Ride (LP Version),200,1543363215796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
1,The Spill Canvas,Logged In,Tegan,F,66,Levine,358.03383,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,The TIde (LP Version),200,1543363420796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
2,Mogwai,Logged In,Tegan,F,67,Levine,571.19302,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Two Rights Make One Wrong,200,1543363778796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
3,Spor,Logged In,Tegan,F,68,Levine,380.34240,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Way Of The Samurai,200,1543364349796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
4,DJ Dizzy,Logged In,Tegan,F,69,Levine,221.15220,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Sexy Bitch,200,1543364729796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80


#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
 - Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
- Create a dataframe, `time_df,` containing the time data for this file

In [15]:
df = df.loc[df['page'] == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Mitch Ryder & The Detroit Wheels,Logged In,Tegan,F,65,Levine,205.03465,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Jenny Take A Ride (LP Version),200,1543363215796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
1,The Spill Canvas,Logged In,Tegan,F,66,Levine,358.03383,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,The TIde (LP Version),200,1543363420796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
2,Mogwai,Logged In,Tegan,F,67,Levine,571.19302,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Two Rights Make One Wrong,200,1543363778796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
3,Spor,Logged In,Tegan,F,68,Levine,380.34240,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Way Of The Samurai,200,1543364349796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
4,DJ Dizzy,Logged In,Tegan,F,69,Levine,221.15220,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Sexy Bitch,200,1543364729796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80


In [16]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

0   2018-11-28 00:00:15.796
1   2018-11-28 00:03:40.796
2   2018-11-28 00:09:38.796
3   2018-11-28 00:19:09.796
4   2018-11-28 00:25:29.796
Name: ts, dtype: datetime64[ns]

In [17]:
time_data = (t.dt.time, t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.weekday)
column_labels = ('time', 'hour', 'day', 'weekofyear', 'month', 'year', 'weekday')

In [18]:
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
time_df.head()

,time,hour,day,weekofyear,month,year,weekday
0,00:00:15.796000,0,28,48,11,2018,2
1,00:03:40.796000,0,28,48,11,2018,2
2,00:09:38.796000,0,28,48,11,2018,2
3,00:19:09.796000,0,28,48,11,2018,2
4,00:25:29.796000,0,28,48,11,2018,2


#### Insert Records into Time Tabl
Implement the `time_table_insert` query in `sql_queries.py` and insert records for the timestamps the log file into the `time` table.

In [19]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [20]:
user_df = df[['userId','firstName','lastName','gender','level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and insert records for the users in the log file into the `users` table. 

In [21]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

## #5: `songplays` Table
#### Extract Data and Songplays Table


#### Insert Records into Songplays Table


In [22]:
for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (index, pd.Timestamp(row.ts).time(), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()